In [ ]:
#set the text file your importing
textfile = "/content/gardians-of-gahool-chapter1.txt"
!pip install openai

In [ ]:
#this will be a automated way is using chatgpt to get the formatting of the quotes used in this story
#works(-----------------)
import openai
import random
import re
import csv

# Initialize the OpenAI API with your API key
openai.api_key = ''  # Make sure you do not expose this key publicly!



def get_random_chunk(filename, chunk_size=1000):
    with open(filename, 'r') as file:
        content = file.read()
        start_index = random.randint(0, len(content) - chunk_size)
        #print(content[start_index: start_index + chunk_size])
        return content[start_index: start_index + chunk_size]

def decipher_quote_symbols(filename, run_counter):
    run_counter = run_counter+1
    print(f"run count: {run_counter}")
    chunk = get_random_chunk(filename)
    query_for_gpt = {
        "messages": [
            {"role": "system", "content": "You are a assistant"},
            {"role": "user", "content": f"Give me a single example of quote in this chunk of text \n\nSample:\n{chunk} only respond with the quote by a character and nothing else."}
        ]
    }

    # Send query to GPT-4
    response = openai.ChatCompletion.create(model="gpt-4", messages=query_for_gpt["messages"])
    response_text = response['choices'][0]['message']['content']
    print(f'Response_text = : {response_text}')
    start_del=response_text[0]
    end_del=response_text[-1]

    print(f"Start Delimiter: {start_del}")
    print(f"End Delimiter: {end_del}")

Copy of
    if run_counter == 5:
      print("ERROR WARNING: Something might be wrong with how your book formats quotes:")
      return start_del, end_del
    if start_del == "'" or end_del == "'":
      print("Warning: one of the deliminators(quote symbols) is ' this could mess things up especially with words like it's and such.")
      print("Re-trying...")
      return decipher_quote_symbols(textfile, run_counter)
    if start_del == "." or end_del == ".":
      print("Warning: one of the deliminators(quote symbols) is ' this could mess things up especially with words like it's and such.")
      print("Re-trying...")
      return decipher_quote_symbols(textfile, run_counter)
    if start_del == " " or end_del == " ":
      print("Warning: one of the deliminators(quote symbols) is a space, this could mess up quote extraction.")
      print(f"start_del: {start_del} end_del: {end_del}")
      print("Re-trying...")
      decipher_quote_symbols(textfile,run_counter)
    return start_del, end_del

# Example usage:
run_counter = 0
start_del, end_del = decipher_quote_symbols(textfile, run_counter)
print(f"Final Start Delimiter: {start_del}")
print(f"Final End Delimiter: {end_del}")

# Read the text file
with open(textfile, 'r') as file:
    text = file.read()

# Use regular expression to find all quotes, including multi-line quotes
quotes = re.findall(f'{start_del}.*?{end_del}', text, re.DOTALL)

# Specify the CSV file to write the data
csv_filename = 'quotes.csv'

# Write quotes and their positions to the CSV file
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Quote', 'Start Location', 'End Location', 'Is Quote'])
    for quote in quotes:
        start = text.find(quote)
        end = start + len(quote)
        is_quote = 'True'
        csv_writer.writerow([quote, start, end, is_quote])

print(f"Quotes and their positions have been written to '{csv_filename}'")


run count: 1
Response_text = : “Now, now, Kludd!” his father admonished. “Don’t be impatient. There’ll be time for branching. Remember, you won’t have your flight feathers for at least another month or more.”
Start Delimiter: “
End Delimiter: ”
Final Start Delimiter: “
Final End Delimiter: ”
Quotes and their positions have been written to 'quotes.csv'


In [ ]:
#DONT RUN YOU PROBS WONT HAVE TO USE THIS METHOD FOR QUOTES
#this is the backup is the quote file generation doesnt work(-----------)
#This will get the quotes from the txt file
import re
import csv
from collections import Counter

def loop_through_symbols():
    # List of common symbols and special characters on an American keyboard
    symbols = [
        "`", "~", "!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "_", "-",
        "+", "=", "{", "}", "[", "]", "|", "\\", ":", ";", "\"", "'", "<", ">",
        ",", ".", "?", "/", " ", "\t", "\n"
    ]

    for symbol in symbols:
        print(f"Press {symbol}")

# Read the text file
with open(textfile, 'r') as file:
    text = file.read()

# Use regular expression to find all non-letter and non-number symbols
non_alnum_symbols = re.findall(r'[^a-zA-Z0-9]', text)

# Count the frequency of each non-alphanumeric symbol
symbol_counts = Counter(non_alnum_symbols)

# Sort the symbols by their frequency in descending order
sorted_symbols = sorted(symbol_counts.items(), key=lambda x: x[1], reverse=True)

# Define a function to check if a combination is a valid quote delimiter
def is_valid_quote(delimiter, text):
    parts = text.split(delimiter)
    return len(parts) % 2 == 0

# Excluded symbols
exclude_symbols = {' ', '\n', ',', '.', '(', ')', }

# Iterate through the sorted symbols and find valid quote delimiters
for symbol, _ in sorted_symbols:
    if symbol not in exclude_symbols:
        if is_valid_quote(symbol, text):
            print(f"Potential quote delimiter: {symbol}")

start_del = input('Enter Start deliminator (if empty):')
end_del = input('Enter End deliminator (if empty):')

# Read the text file
with open(textfile, 'r') as file:
    text = file.read()

# Use regular expression to find all quotes, including multi-line quotes
quotes = re.findall(f'{start_del}.*?{end_del}', text, re.DOTALL)

# Specify the CSV file to write the data
csv_filename = 'quotes.csv'

# Write quotes and their positions to the CSV file
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Quote', 'Start Location', 'End Location', 'Is Quote'])
    for quote in quotes:
        start = text.find(quote)
        end = start + len(quote)
        is_quote = 'True'
        csv_writer.writerow([quote, start, end, is_quote])

print(f"Quotes and their positions have been written to '{csv_filename}'")


Potential quote delimiter: “
Potential quote delimiter: !
Potential quote delimiter: /
Enter Start deliminator (if empty):"
Enter End deliminator (if empty):"
Quotes and their positions have been written to 'quotes.csv'


In [ ]:
import csv
#(---------------)
#create the non quotes csv file
# Step 1: Read the quotes.csv file and get all start and end locations.
locations = []
with open('quotes.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # skip the header
    for row in csvreader:
        start_location = int(row[1])  # assuming start location is in the first column
        end_location = int(row[2])    # assuming end location is in the second column
        locations.append((start_location, end_location))

locations.sort(key=lambda x: x[0])  # sort the locations based on start_location

results = []

# Step 2: Open Alice_in_wonderland.txt and extract the required snippets.
with open(textfile, 'r') as file:
    content = file.read()
    prev_end_location = 0  # initialize with 0
    for start_location, end_location in locations:
        snippet = content[prev_end_location:start_location].strip()
        if snippet:  # check if the snippet is not empty
            results.append((snippet, prev_end_location, start_location, 'False', 'Narrator'))  # appending 'False' and 'Narrator'
        prev_end_location = end_location

    # Collect anything after the last end location to the end of the file
    snippet = content[prev_end_location:].strip()
    if snippet:
        results.append((snippet, prev_end_location, len(content), 'False', 'Narrator'))  # appending 'False' and 'Narrator'

# Step 3: Write the results to a new CSV file.
with open('non_quotes.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Text', 'Start Location', 'End Location', 'Is Quote', 'Speaker'])  # added 'Speaker' to the header
    for row in results:
        csvwriter.writerow(row)
        print(row)


('The Capture\nGuardians of Ga’hoole\nBy\nKathryn Lasky\n\nCHAPTER ONE\n\nA Nest Remembered\n\n\n\nNoctus, can you spare a bit more down, darling? I think our third little one is about to arrive. That egg is beginning to crack.”', 0, 217, 'False', 'Narrator')
('sighed Kludd.', 229, 245, 'False', 'Narrator')
('his father said. There was an edge to his voice.', 321, 372, 'False', 'Narrator')
('His mother sighed the low soft whistle Barn Owls sometimes used.', 384, 451, 'False', 'Narrator')
('Soren peeped up.', 471, 490, 'False', 'Narrator')
('Kludd turned to Soren, his younger brother.', 527, 572, 'False', 'Narrator')
('Maybe, Soren thought to himself, they would be better than brothers. Kludd seemed to have resented him since the moment he had first hatched.', 605, 750, 'False', 'Narrator')
('Kludd said dully. Branching was the first step, literally, toward flight. The young owlets would begin by hopping from branch to branch and flapping their wings.', 833, 997, 'False', 'Narrator')
('

In [ ]:
#this will find out how many rows there are in the quotes csv file
import csv

# Replace 'your_file.csv' with the name of your CSV file.
file_name = 'quotes.csv'

# Initialize a row counter.
row_count = 0

# Open the CSV file.
with open(file_name, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)

    # Loop through each row in the CSV file.
    for row in csv_reader:
        row_count += 1

# Print the total number of rows.
print(f"Total number of rows in '{file_name}': {row_count}")


Total number of rows in 'quotes.csv': 106


In [ ]:
#this code overcomes the limits set by openai by adding a rest function
#This is working best version------------------
#this will be identifying the speakers for only the quotes in the book and putting them in the csv file
import openai
import pandas as pd
import time

# Initialize your API key
openai.api_key = ''

def extract_sentence(start, end, filename=textfile):
    with open(filename, 'r') as f:
        content = f.read()
    print(content[start:end])

    #return content[start:end]
    sentence = content[start:end]
    return sentence

def extract_context(sentence, filename=textfile):
    with open(filename, 'r') as f:
        content = f.read()
    start_index = content.find(sentence)

    if start_index == -1:
        return None  # sentence not found

    left_index = max(0, start_index - 500)
    right_index = min(len(content), start_index + len(sentence) + 500)

    return content[left_index:right_index]

def ask_openai(context, sentence, names):
    message = {
        "role": "system",
        "content": "You are a helpful assistant that determines which character is speaking in a story you answer with only the name and nothing else"
    }
    message_user = {
        "role": "user",
        "content": f"{context} :who said {sentence} in the text i gave, here is a list of names of characters you have identified so far{names}"
    }

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[message, message_user]
    )
    return response.choices[0].message['content'].strip()

def main():
    df = pd.read_csv('quotes.csv')  # Assuming your CSV has 'Start Location', 'End Location', and 'Quote' columns
    responses = []
    names = []


    request_counter = 0  # Initialize the counter to 0
    num_of_wanted_requests = row_count

    # Limiting to the first numer of wanted rows
    for index, row in df.head(num_of_wanted_requests).iterrows():
        sentence = extract_sentence(row['Start Location'], row['End Location'])
        context = extract_context(sentence)
        response = ask_openai(context, sentence, names)
        responses.append(response)
        if names.count(response) > 0:
          print(f'{response} already exists in names list')
        else:
          print(f'{response} does not exist in names list adding...')
          names.append(response)
        print(names)

        request_counter += 1
        print(f"request counter: {request_counter}")

        if request_counter % 20 == 0:
            print("sleeping for 60 seconds...")
            time.sleep(60)  # Wait for 60 seconds




    # Update only the relevant rows with responses
    df.loc[df.index[:num_of_wanted_requests], 'Speaker'] = responses

    df.to_csv('quotes.csv', index=False)
# Call main
main()


“Not again!”
Kludd does not exist in names list adding...
['Kludd']
request counter: 1
“What do you mean, Kludd, not again? Don’t you want another little brother?”
Noctus does not exist in names list adding...
['Kludd', 'Noctus']
request counter: 2
“Or sister?”
Noctus already exists in names list
['Kludd', 'Noctus']
request counter: 3
“I’d like a sister,”
Soren does not exist in names list adding...
['Kludd', 'Noctus', 'Soren']
request counter: 4


In [ ]:
#works(--------------------)
#this will create the book.csv file output
import csv

# Step 1: Read the contents of quotes.csv and non_quotes.csv into two lists.
quotes = []
with open('quotes.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # skip the header
    for row in csvreader:
        text = row[0]
        start_location = int(row[1])
        end_location = int(row[2])
        speaker = row[4]  # Assuming "Speaker" is in the 4th column
        quotes.append((text, start_location, end_location, speaker, 'True'))

results = []
with open('non_quotes.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # skip the header
    for row in csvreader:
        text = row[0]
        start_location = int(row[1])
        end_location = int(row[2])
        speaker = row[4]  # Assuming "Speaker" is in the 4th column
        results.append((text, start_location, end_location, speaker, 'False'))

# Step 2: Merge and sort the two lists by start location.
combined = quotes + results
combined.sort(key=lambda x: x[1])  # sort based on start location

# Step 3: Write the sorted list to book.csv.
with open('book.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Text', 'Start Location', 'End Location', 'Speaker', 'Is Quote'])
    for row in combined:
        csvwriter.writerow(row)
